In [2]:
import pandas as pd
from bidi.algorithm import get_display
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [5]:
df = pd.read_csv("/Users/Lana/Desktop/Misk_DSI_capstone/abjjad.csv",encoding="utf-8-sig")

In [6]:
df.head(4)

,Unnamed: 0,Book_title,Author,Cover_url,genres,descriptions,cover_name,description_length,genre_count
0,0,أحببتك أكثر مما ينبغي,أثير عبد الله النشمي,https://cdn.abjjad.com/pub/369c6a19-3ed7-4d0f-...,"['روايات وقصص', 'روايات رومانسية', 'روايات خيا...",تنقل الروايه بصدق عميق يدور اعماق المراه حاله ...,369c6a19-3ed7-4d0f-a449-6f8173cf6abd-192X290.png,157,3
1,1,هكذا خُلقت,محمد حسين هيكل,https://cdn.abjjad.com/pub/cbb863f2-e58c-4fa5-...,"['روايات وقصص', 'روايات اجتماعية', 'روايات واق...",روايه واقعيه تركتها يدي المءلف امراه غريبه غام...,cbb863f2-e58c-4fa5-80f3-2cba733f258f-192X290.png,110,3
2,2,قواعد العشق الأربعون,إليف شافاق,https://cdn.abjjad.com/pub/44a22888-7eea-4714-...,"['كتب الأدب', 'أدب عالمي مترجم', 'روايات وقصص'...",بلغت بطله الروايه ايلا الزوجه التعيسه سن الارب...,44a22888-7eea-4714-9c54-3dbf463eccdd-192X290.png,46,8
3,3,الأرواح المتمردة,جبران خليل جبران,https://cdn.abjjad.com/pub/d95eb7b7-ebfd-47c1-...,"['روايات وقصص', 'قصص قصيرة', 'روايات اجتماعية'...",الارواح المتمرده كتاب صدر لاول مره مدينه نيويو...,d95eb7b7-ebfd-47c1-a1a0-7f709c45bfdc-192X290.png,76,5


In [ ]:
##Converting genres to Features
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df['genres'])

y = multilabel_binarizer.transform(df['genres'])

vect = TfidfVectorizer(max_df=0.8, max_features=4000)
# transform target variable
#y = multilabel_binarizer.transform(D_FMT['genres'])

# split dataset 
xtrain, xtest, ytrain, ytest = train_test_split(df['descriptions'], y, test_size=0.2, random_state=9)

# TF-IDF 
xtrain_tfidf = vect.fit_transform(xtrain)
xtest_tfidf = vect.transform(xtest)

## MODEL
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

# fit model 
clf.fit(xtrain_tfidf, ytrain)

# make predictions for validation set
y_pred = clf.predict(xtest_tfidf)
print(y_pred[5])

print(multilabel_binarizer.inverse_transform(y_pred)[3])

y_pred_prob = clf.predict_proba(xtest_tfidf)


threshold = 0.15 # threshold value
y_pred_new = (y_pred_prob >= threshold).astype(int)

# evaluate performance
print(f1_score(ytest, y_pred_new, average="micro"))
print(accuracy_score(ytest, y_pred_new))